In [ ]:
!pip install yfinance

In [ ]:
# IMPORTS
import numpy as np
import pandas as pd

#Fin Data Sources
import yfinance as yf
import pandas_datareader as pdr

#Data viz
import plotly.graph_objs as go
import plotly.express as px

import time
from datetime import date

# for graphs
import matplotlib.pyplot as plt

# for Data ingestion and manipulation
import pandas as pd
import requests

### Question 3: Is Growth Concentrated in the Largest Stocks?

**Get the share of days (percentage as int) when Large Stocks outperform (growth_7d - growth over 7 periods back) the Largest stocks?**


Reuse [Code Snippet 5] to obtain OHLCV stats for 33 stocks
for 10 full years of data (2014-01-01 to 2023-12-31):


In [ ]:
# GEt the LARGEST STOCK companies
US_STOCKS = ['MSFT', 'AAPL', 'GOOG', 'NVDA', 'AMZN', 'META', 'BRK-B', 'LLY', 'AVGO','V', 'JPM'] #11

EU_STOCKS = ['NVO','MC.PA', 'ASML', 'RMS.PA', 'OR.PA', 'SAP', 'ACN', 'TTE', 'SIE.DE','IDEXY','CDI.PA'] #11

INDIA_STOCKS = ['RELIANCE.NS','TCS.NS','HDB','BHARTIARTL.NS','IBN','SBIN.NS','LICI.NS','INFY','ITC.NS','HINDUNILVR.NS','LT.NS'] #11

LARGEST_STOCKS = US_STOCKS + EU_STOCKS + INDIA_STOCKS
print('Num stocks in LARGEST STOCKS: ',len(LARGEST_STOCKS))

NEW_US = ['TSLA','WMT','XOM','UNH','MA','PG','JNJ','MRK','HD','COST','ORCL'] #11

NEW_EU = ['PRX.AS','CDI.PA','AIR.PA','SU.PA','ETN','SNY','BUD','DTE.DE','ALV.DE','MDT','AI.PA','EL.PA'] #12

NEW_INDIA = ['BAJFINANCE.NS','MARUTI.NS','HCLTECH.NS','TATAMOTORS.NS','SUNPHARMA.NS','ONGC.NS','ADANIENT.NS','ADANIENT.NS','NTPC.NS','KOTAKBANK.NS','TITAN.NS'] #11

LARGE_STOCKS = NEW_EU + NEW_US + NEW_INDIA
print('Num stocks in LARGE STOCKS: ',len(LARGE_STOCKS))

In [ ]:
# https://companiesmarketcap.com/usa/largest-companies-in-the-usa-by-market-cap/
US_STOCKS = ['MSFT', 'AAPL', 'GOOG', 'NVDA', 'AMZN', 'META', 'BRK-B', 'LLY', 'AVGO','V', 'JPM']

# You're required to add EU_STOCKS and INDIA_STOCS
# https://companiesmarketcap.com/european-union/largest-companies-in-the-eu-by-market-cap/
EU_STOCKS = ['NVO','MC.PA', 'ASML', 'RMS.PA', 'OR.PA', 'SAP', 'ACN', 'TTE', 'SIE.DE','IDEXY','CDI.PA']

# https://companiesmarketcap.com/india/largest-companies-in-india-by-market-cap/
INDIA_STOCKS = ['RELIANCE.NS','TCS.NS','HDB','BHARTIARTL.NS','IBN','SBIN.NS','LICI.NS','INFY','ITC.NS','HINDUNILVR.NS','LT.NS']

ALL_TICKERS = US_STOCKS  + EU_STOCKS + INDIA_STOCKS
ALL_TICKERS

print(ALL_TICKERS)
print(len(ALL_TICKERS))

In [ ]:
len(set(LARGEST_STOCKS))
len(set(LARGE_STOCKS))

In [ ]:
  historyPrices = yf.download(tickers = 'BUD',
                     period = "max",
                     start="2014-01-01", end="2023-12-31",
                     interval = "1d")

In [ ]:
historyPrices

In [ ]:
import time

stocks_df = pd.DataFrame({'A' : []})

for i,ticker in enumerate(set(ALL_TICKERS)):
  print(i,ticker)

  # Work with stock prices
  historyPrices = yf.download(tickers = ticker,
                     period = "max",
                     #start="2014-01-01", end="2023-12-31",
                     interval = "1d")

  # generate features for historical prices, and what we want to predict
  historyPrices['Ticker'] = ticker
  historyPrices['Year']= historyPrices.index.year
  historyPrices['Month'] = historyPrices.index.month
  historyPrices['Weekday'] = historyPrices.index.weekday
  historyPrices['Date'] = historyPrices.index.date

  # historical returns
  #for i in [1,3,7]:
  i= 5
  #historyPrices[f'Adj Close {i}'] = historyPrices['Adj Close'].shift(i)
  historyPrices['growth_'+str(i)+'d'] = historyPrices['Adj Close'] / historyPrices['Adj Close'].shift(i)
  historyPrices[f'Adj Close {i}'] = historyPrices['Adj Close'].shift(-i)
  historyPrices[f'Date {i}'] = historyPrices['Date'].shift(i)
  historyPrices['quantity'] = 1000.0/ historyPrices['Adj Close']
  historyPrices['invest'] = historyPrices['Adj Close']*historyPrices['quantity']
  historyPrices['retinvest'] = historyPrices[f'Adj Close {i}']*historyPrices['quantity']
  historyPrices['profit'] = historyPrices['retinvest']-historyPrices['invest']

  #historyPrices['growth_future_5d'] = historyPrices['Adj Close'].shift(-5) / historyPrices['Adj Close']

  # Technical indicators
  # SimpleMovingAverage 10 days and 20 days
  #historyPrices['SMA10']= historyPrices['Close'].rolling(10).mean()
  #historyPrices['SMA20']= historyPrices['Close'].rolling(20).mean()
  #historyPrices['growing_moving_average'] = np.where(historyPrices['SMA10'] > historyPrices['SMA20'], 1, 0)
  #historyPrices['high_minus_low_relative'] = (historyPrices.High - historyPrices.Low) / historyPrices['Adj Close']

  # 30d rolling volatility : https://ycharts.com/glossary/terms/rolling_vol_30
  #historyPrices['volatility'] =   historyPrices['Adj Close'].rolling(30).std() * np.sqrt(252)

  # what we want to predict
  #historyPrices['is_positive_growth_5d_future'] = np.where(historyPrices['growth_future_5d'] > 1, 1, 0)

  # sleep 1 sec between downloads - not to overload the API server
  time.sleep(1)


  if stocks_df.empty:
    stocks_df = historyPrices
  else:
    stocks_df = pd.concat([stocks_df, historyPrices], ignore_index=True)


In [ ]:
#stocks_df_large= stocks_df
stocks_df_largest= stocks_df

In [ ]:
stocks_df.head()

In [ ]:
stocks_df.info()

In [ ]:
(
    stocks_df_largest
    #.loc[(stocks_df['Year']==2014) & (stocks_df['Month']==1)]
    #.loc[(stocks_df['Ticker']=='BAJFINANCE.NS')]
    .loc[stocks_df_largest['Date']==date(2014,1,1)]
    #.sort_values(by=['Date'], ascending=True)
    [['Adj Close','Adj Close 7','growth_7d']]
    .agg('mean')
)

In [ ]:
stocks_df_large = (
    stocks_df_large
    #.loc[(stocks_df['Year']==2014) & (stocks_df['Month']==1)]
    #.loc[(stocks_df['Ticker']=='BAJFINANCE.NS')]
    .groupby('Date')
    .agg(
        avg_growth_7d=("growth_7d", "mean"),
    )
)

stocks_df_largest = (
    stocks_df_largest
    #.loc[(stocks_df['Year']==2014) & (stocks_df['Month']==1)]
    #.loc[(stocks_df['Ticker']=='BAJFINANCE.NS')]
    .groupby('Date')
    .agg(
        avg_growth_7d=("growth_7d", "mean"),
    )
)

In [ ]:
stocks_df_large.info()

In [ ]:
(
    stocks_df_large
    [(stocks_df_large.index >= date(2014,1,1)) & (stocks_df_large.index <= date(2023,12,31))]
    #[stocks_df_large.index >= date(2014,1,1)]
    .join(stocks_df_largest[(stocks_df_largest.index >= date(2014,1,1)) & (stocks_df_largest.index <= date(2023,12,31))]
                    , on='Date', how='left', lsuffix='_large', rsuffix='_largest')
    .loc[lambda df: df['avg_growth_7d_large'] > df['avg_growth_7d_largest']]
)

In [ ]:
1212/2595

### Question 4: Trying Another Technical Indicators strategy

**What's the total gross profit (in THOUSANDS of $) you'll get from trading on CCI (no fees assumption)?**


First, run the entire Colab to obtain the full DataFrame of data (after [Code Snippet 9]), and truncate it to the last full 10 years of data (2014-01-01 to 2023-12-31).
If you encounter any difficulties running the Colab - you can download it using this [link](https://drive.google.com/file/d/1m3Qisfs2XfWk6Sw_Uk5kHLWqwQ0q8SKb/view?usp=sharing).

Let's assume you've learned about the awesome **CCI indicator** ([Commodity Channel Index](https://www.investopedia.com/terms/c/commoditychannelindex.asp)), and decided to use only it for your operations.

You defined the "defensive" value of a high threshould of 200, and you trade only on Fridays (`Date.dt.dayofweek()==4`).

That is, every time you see that CCI is >200 for any stock (out of those 33), you'll invest $1000 (each record when CCI>200) at Adj.Close price and hold it for 1 week (5 trading days) in order to sell at the Adj. Close price.

What's the expected gross profit (no fees) that you get in THOUSANDS $ (closest integer value) over many operations in 10 years?
One operation calculations: if you invested $1000 and received $1010 in 5 days - you add $10 to gross profit, if you received $980 - add -$20 to gross profit.
You need to sum these results over all trades (460 times in 10 years).

Additional:
  * Add an approximate fees calculation over the 460 trades from this calculator https://www.degiro.ie/fees/calculator (Product:"Shares, USA and Canada;" Amount per transaction: "1000 EUR"; Transactions per year: "460")
  * are you still profitable on those trades?


In [ ]:
import pandas as pd
import pyarrow
import fastparquet

### First approach: Recreate the dataset

In [ ]:
# https://companiesmarketcap.com/usa/largest-companies-in-the-usa-by-market-cap/
US_STOCKS = ['MSFT', 'AAPL', 'GOOG', 'NVDA', 'AMZN', 'META', 'BRK-B', 'LLY', 'AVGO','V', 'JPM']

# You're required to add EU_STOCKS and INDIA_STOCS
# https://companiesmarketcap.com/european-union/largest-companies-in-the-eu-by-market-cap/
EU_STOCKS = ['NVO','MC.PA', 'ASML', 'RMS.PA', 'OR.PA', 'SAP', 'ACN', 'TTE', 'SIE.DE','IDEXY','CDI.PA']

# https://companiesmarketcap.com/india/largest-companies-in-india-by-market-cap/
INDIA_STOCKS = ['RELIANCE.NS','TCS.NS','HDB','BHARTIARTL.NS','IBN','SBIN.NS','LICI.NS','INFY','ITC.NS','HINDUNILVR.NS','LT.NS']

ALL_TICKERS = US_STOCKS  + EU_STOCKS + INDIA_STOCKS
ALL_TICKERS

print(ALL_TICKERS)
print(len(ALL_TICKERS))

Create a dataframe with the tickers defined and the OHLCV values

In [ ]:
import time

stocks_df = pd.DataFrame({'A' : []})

for i,ticker in enumerate(set(ALL_TICKERS)):
  print(i,ticker)

  # Work with stock prices
  historyPrices = yf.download(tickers = ticker,
                     period = "max",
                     #start="2014-01-01", end="2023-12-31",
                     interval = "1d")

  # generate features for historical prices, and what we want to predict
  historyPrices['Ticker'] = ticker
  historyPrices['Year']= historyPrices.index.year
  historyPrices['Month'] = historyPrices.index.month
  historyPrices['Weekday'] = historyPrices.index.weekday
  historyPrices['Date'] = historyPrices.index.date

  # historical returns
  #for i in [1,3,7]:
  i= 5
  #historyPrices[f'Adj Close {i}'] = historyPrices['Adj Close'].shift(i)
  #historyPrices['growth_'+str(i)+'d'] = historyPrices['Adj Close'] / historyPrices['Adj Close'].shift(i)
  historyPrices[f'Adj Close {i}'] = historyPrices['Adj Close'].shift(-i)
  historyPrices[f'Date {i}'] = historyPrices['Date'].shift(i)
  historyPrices['quantity'] = 1000.0/ historyPrices['Adj Close']
  historyPrices['invest'] = historyPrices['Adj Close']*historyPrices['quantity']
  historyPrices['retinvest'] = historyPrices[f'Adj Close {i}']*historyPrices['quantity']
  historyPrices['profit'] = historyPrices['retinvest']-historyPrices['invest']


  # sleep 1 sec between downloads - not to overload the API server
  time.sleep(1)


  if stocks_df.empty:
    stocks_df = historyPrices
  else:
    stocks_df = pd.concat([stocks_df, historyPrices], ignore_index=True)


Calculate CCI values

In [ ]:
talib_momentum_cci = talib.CCI(stocks_df.High.values, stocks_df.Low.values, stocks_df.Close.values, timeperiod=14)

In [ ]:
stocks_df['cci']=talib_momentum_cci

In [ ]:
stocks_df

In [ ]:
profit=(
    stocks_df
    #.loc[(df['Date']>='2014-01-01') & (df['Date']<='2014-20-01')&(df['Weekday']==4)]
    #.loc[(df['Date']>='2014-01-01') & (df['Date']<='2023-12-31')] #&(df['Weekday']==4)] #& (df['cci']>200)]
    [['Date','Weekday', 'Ticker','cci','Adj Close','quantity','invest','retinvest','profit']]
    .loc[(stocks_df['Date']>=date(2014,1,1)) & (stocks_df['Date']<=date(2023,12,31))]
    .loc[(stocks_df['Weekday']==4) & (stocks_df['cci']>200)] #Date.dt.dayofweek()==4
    #.loc[(df['Date'].dt.dayofweek()==4) & (df['cci']>200)]
    #[['profit']].sum()
    .sort_values(['Date','Ticker'])
)

In [ ]:
profit['profit'].sum()

In [ ]:
np.mean([1,2, 3,4,5,6])

In [ ]:
np.nan

In [ ]:
np.mean([1,2, 3,4,5,6,6])

In [ ]:
index_a = [1,1,2,2,3,3]
index_b = [0,1,1,0,1,2]
index = pd.MultiIndex.from_arrays([index_a,index_b], names=('a','b'))
np.random.seed(3)
df1 = pd.DataFrame(np.random.rand(4,), index = [1,2,3,3], columns=['p'])
df2 = pd.DataFrame(np.random.rand(6,), index=index, columns=['p'])

In [ ]:
df1

In [ ]:
df2

In [ ]:
import time

stocks_df = pd.DataFrame({'A' : []})

for i,ticker in enumerate(set(ALL_TICKERS)):
  print(i,ticker)

  # Work with stock prices
  historyPrices = yf.download(tickers = ticker,
                     period = "max",
                     #start="2014-01-01", end="2023-12-31",
                     interval = "1d")

  # generate features for historical prices, and what we want to predict
  historyPrices['Ticker'] = ticker
  historyPrices['Year']= historyPrices.index.year
  historyPrices['Month'] = historyPrices.index.month
  historyPrices['Weekday'] = historyPrices.index.weekday
  historyPrices['Date'] = historyPrices.index.date

  # historical returns
  #for i in [1,3,7]:
  i= 5
  #historyPrices[f'Adj Close {i}'] = historyPrices['Adj Close'].shift(i)
  historyPrices['growth_'+str(i)+'d'] = historyPrices['Adj Close'] / historyPrices['Adj Close'].shift(i)
  historyPrices[f'Adj Close {i}'] = historyPrices['Adj Close'].shift(-i)
  historyPrices[f'Date {i}'] = historyPrices['Date'].shift(i)
  historyPrices['quantity'] = 1000.0/ historyPrices['Adj Close']
  historyPrices['invest'] = historyPrices['Adj Close']*historyPrices['quantity']
  historyPrices['retinvest'] = historyPrices[f'Adj Close {i}']*historyPrices['quantity']
  historyPrices['profit'] = historyPrices['retinvest']-historyPrices['invest']

  #historyPrices['growth_future_5d'] = historyPrices['Adj Close'].shift(-5) / historyPrices['Adj Close']

  # Technical indicators
  # SimpleMovingAverage 10 days and 20 days
  #historyPrices['SMA10']= historyPrices['Close'].rolling(10).mean()
  #historyPrices['SMA20']= historyPrices['Close'].rolling(20).mean()
  #historyPrices['growing_moving_average'] = np.where(historyPrices['SMA10'] > historyPrices['SMA20'], 1, 0)
  #historyPrices['high_minus_low_relative'] = (historyPrices.High - historyPrices.Low) / historyPrices['Adj Close']

  # 30d rolling volatility : https://ycharts.com/glossary/terms/rolling_vol_30
  #historyPrices['volatility'] =   historyPrices['Adj Close'].rolling(30).std() * np.sqrt(252)

  # what we want to predict
  #historyPrices['is_positive_growth_5d_future'] = np.where(historyPrices['growth_future_5d'] > 1, 1, 0)

  # sleep 1 sec between downloads - not to overload the API server
  time.sleep(1)


  if stocks_df.empty:
    stocks_df = historyPrices
  else:
    stocks_df = pd.concat([stocks_df, historyPrices], ignore_index=True)


In [ ]:
stocks_df

In [ ]:
from datetime import date

In [ ]:
stocks_df_2014_2023 = (
    stocks_df
    .loc[(stocks_df['Date']>=date(2014,1,1)) & (stocks_df['Date']<=date(2023,12,31))]
    #.loc[(stocks_df['Weekday']==4)] #Date.dt.dayofweek()==4
    #.loc[(stocks_df['Date']==date(2014,5,30))]

)

In [ ]:
stocks_df_2014_2023

In [ ]:
# https://stackoverflow.com/questions/49648391/how-to-install-ta-lib-in-google-colab
# Update (apr 2023): Colab is now Python 3.10
# there are some other (older) ways to install talib

url = 'https://anaconda.org/conda-forge/libta-lib/0.4.0/download/linux-64/libta-lib-0.4.0-h166bdaf_1.tar.bz2'
!curl -L $url | tar xj -C /usr/lib/x86_64-linux-gnu/ lib --strip-components=1
url = 'https://anaconda.org/conda-forge/ta-lib/0.4.19/download/linux-64/ta-lib-0.4.19-py310hde88566_4.tar.bz2'
!curl -L $url | tar xj -C /usr/local/lib/python3.10/dist-packages/ lib/python3.10/site-packages/talib --strip-components=3
import talib

In [ ]:
talib_momentum_cci = talib.CCI(stocks_df.High.values, stocks_df.Low.values, stocks_df.Close.values, timeperiod=14)

In [ ]:
len(talib_momentum_cci)

In [ ]:
stocks_df['cci']=talib_momentum_cci

In [ ]:
stocks_df

In [ ]:
from datetime import date

In [ ]:
profit=(
    stocks_df
    #.loc[(df['Date']>='2014-01-01') & (df['Date']<='2014-20-01')&(df['Weekday']==4)]
    #.loc[(df['Date']>='2014-01-01') & (df['Date']<='2023-12-31')] #&(df['Weekday']==4)] #& (df['cci']>200)]
    [['Date','Weekday', 'Ticker','cci','Adj Close']]
    .assign(adj_close_5= lambda df: df['Adj Close'].shift(-5))
    .assign(quantity= lambda df: 1000/df['Adj Close'])
    .assign(invest= lambda df: df['Adj Close']*df['quantity'])
    .assign(retinvest= lambda df: df['adj_close_5']*df['quantity'])
    #.assign(profit= lambda df: df['retinvest']-1000.0)
    .assign(profit= lambda df: df['retinvest']-df['invest'])
    #.loc[df['cci']>200]
    #.assign(profit= lambda df: (df['adj_close_5']*df['quantity'])-1000.0)
    #.assign(dayofweek= lambda df: df['Date'].dt.dayofweek())
    .loc[(stocks_df['Date']>=date(2014,1,1)) & (stocks_df['Date']<=date(2023,12,31))]
    .loc[(stocks_df['Weekday']==4) & (stocks_df['cci']>200)] #Date.dt.dayofweek()==4
    #.loc[(df['Date'].dt.dayofweek()==4) & (df['cci']>200)]
    [['profit']].sum()
)

In [ ]:
np.rint(profit/1000)

Additional:

- Add an approximate fees calculation over the 460 trades from this calculator https://www.degiro.ie/fees/calculator (Product:"Shares, USA and Canada;" Amount per transaction: "1000 EUR"; Transactions per year: "460")

- are you still profitable on those trades?

DEGIRO costs

2,072.50

Commission fee

€460.00

Handling fees

€460.00

Connectivity fee

€2.50

Currency cost

€1,150.00*

Estimated bid-ask spread cost

€138.001

Total costs

€2,210.50

In [ ]:
(
    stocks_df
    #.loc[(df['Date']>='2014-01-01') & (df['Date']<='2014-20-01')&(df['Weekday']==4)]
    #.loc[(df['Date']>='2014-01-01') & (df['Date']<='2023-12-31')] #&(df['Weekday']==4)] #& (df['cci']>200)]
    [['Date','Weekday', 'Ticker','cci','Adj Close']]
    .assign(adj_close_5= lambda df: df['Adj Close'].shift(-5))
    .assign(quantity= lambda df: 1000/df['Adj Close'])
    .assign(invest= lambda df: df['Adj Close']*df['quantity'])
    .assign(retinvest= lambda df: df['adj_close_5']*df['quantity'])
    #.assign(profit= lambda df: df['retinvest']-1000.0)
    .assign(profit= lambda df: df['retinvest']-df['invest'])
    #.loc[df['cci']>200]
    #.assign(profit= lambda df: (df['adj_close_5']*df['quantity'])-1000.0)
    #.assign(dayofweek= lambda df: df['Date'].dt.dayofweek())
    .loc[(stocks_df['Date']>=date(2014,1,1)) & (stocks_df['Date']<=date(2023,12,31))]
    #.loc[(stocks_df['Weekday']==4) & (stocks_df['cci']>200)] #Date.dt.dayofweek()==4
    .loc[stocks_df['Date']==date(2023,12,15)]
    #[['profit']].sum()
    .sort_values('Ticker')
)